In [11]:
# imports
import atc_dataloader, atc_model
import torch
from torch.utils.data import DataLoader
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from atc_loss import PredictionLoss_COS_MSE, PredictionLoss_BOX_Wise
import time

In [2]:
# set the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
def transform_vector_to_integers(pred):
    # //////////////////////////////////////////////////////////
    # CURRENTLY NOT USED
    # //////////////////////////////////////////////////////////
    # get the max value, that will serve as reference - where is the max, there is supposed to be 1 (or other integer, depends...)
    # for now, everything until 1.3 will be treated as 1, until 2.3 as two etc.
    
    # lambda to round function from the specific treshold defined in range [.0;1.0)
    my_treshold = 0.3
    tr = np.min([my_treshold, pred.max()*0.9])

    myround = np.vectorize(lambda x, treshold=tr: np.floor(x) if x < (np.floor(x) + treshold) else np.ceil(x))

    result = np.apply_along_axis(func1d=myround, axis=0,arr=pred)
    return result

def evaluate(model, data_loader, device, criterion):
    """
    function used to evaluate the model, return loss and accuracy
    """
    model.eval()
    
    test_loss = []
    with torch.no_grad():
        for  _, batch_in_vectors, batch_out_vectors in data_loader:
            pred = model(batch_in_vectors.to(device))
            loss = criterion(pred, batch_out_vectors.to(device))
            test_loss.append(loss.item())
    
    return np.mean(test_loss)

# def evaluate_real_similarity(dl_test,model,device):
    

In [4]:
def real_vector_similarity(vec1, vec2, tolerance=0.1):
    # Convert to numpy arrays for element-wise comparison
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    
    # Compute the absolute difference between corresponding elements
    difference = np.abs(vec1 - vec2)
    print(difference)
    # Check if all differences are within the given tolerance
    return np.all(difference <= tolerance)

real_vector_similarity([[1.0, 2.0, 3.0],[5.09, 6.0, 7.0]], [[1.0, 2.0, 3.0],[5.3, 6.0, 7.0]], tolerance=0.1)

[[0.   0.   0.  ]
 [0.21 0.   0.  ]]


False

In [16]:
def train_model(model, device, train_dl, test_dl, criterion, optimizer, epochs=10, save_model_last_epoch=True, model_path='.'):
    
    # loss that will be calculated after each epoch for both test and train set 
    train_loss_overall = []
    test_loss_overall = [] 
    
    for epoch in range(epochs):
        epoch_loss = []
        model.train()  # Set the model to training mode
        
        # Wrap dataloader with tqdm for progress visualization
        for _, batch_in_vectors, batch_out_vectors in tqdm(train_dl, desc=f'Epoch {epoch + 1}/{epochs}', unit='batch'):
            # Convert data to tensors and move to the correct device
            batch_in_vectors = torch.tensor(batch_in_vectors, dtype=torch.float32, requires_grad=True).to(device)
            batch_out_vectors = torch.tensor(batch_out_vectors, dtype=torch.float32).to(device)
            
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(batch_in_vectors)
            
            # Calculate loss
            loss = criterion(outputs, batch_out_vectors)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Accumulate loss
            epoch_loss.append(loss.item())

        if (epoch % 20 == 0):
            torch.save(model.state_dict(), f'models/model_betternn_epoch_{epoch}.pth')
        
        # start evaluation and append test loss
        tmp_test_loss = evaluate(model, test_dl, device, criterion)
        test_loss_overall.append(tmp_test_loss)
        
        # Calculate average train loss for the epoch
        avg_train_loss = np.mean(epoch_loss)
        train_loss_overall.append(avg_train_loss) # add it to the loss over all epochs
        
        # Print loss (both train and test) for the current epoch
        print(f"Epoch [{epoch+1}/{epochs}], Train loss: {avg_train_loss:.4f}, Test loss: {tmp_test_loss:.4f}")

    # ====================================
    # AFTER TRAIN ========================
    # ====================================
    
    # save the model after the last epoch if set
    # if (save_model_last_epoch):
    #     torch.save(model.state_dict(), f'{model_path}/model_last_epoch_{time.time_ns()}.pth')
        
    # print overall loss
    print('Overall train loss: ', train_loss_overall)
    print('Overall test loss: ', test_loss_overall)
    # plot graph
    plt.figure()
    plt.plot(np.arange(0,epochs,1),train_loss_overall, label='Train loss') # train loss over all epochs
    plt.plot(np.arange(0,epochs,1),test_loss_overall, label='Test loss') # test loss over all epochs
    plt.title('Loss function')
    plt.show()
    
    return train_loss_overall, test_loss_overall

### Train part

In [6]:
# load data
TRAIN_IN="data/train_in.csv"
TRAIN_OUT="data/train_out.csv"
# TRAIN_IN='data/sample_in.csv'
# TRAIN_OUT='data/sample_out.csv'
data_train = atc_dataloader.ATCDataset_v2(TRAIN_IN, TRAIN_OUT)

In [7]:
TEST_IN="data/test_in.csv"
TEST_OUT="data/test_out.csv"
# TEST_IN='data/sample_in.csv'
# TEST_OUT='data/sample_out.csv'
data_test = atc_dataloader.ATCDataset_v2(TEST_IN, TEST_OUT)

In [8]:
model = atc_model.BetterNN().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, weight_decay=0.005)
criterion = PredictionLoss_BOX_Wise().to(device)
dl_train = DataLoader(data_train, batch_size=32, shuffle=True)  
dl_test = DataLoader(data_test, batch_size=32, shuffle=False)

In [ ]:
train_loss, test_loss = train_model(model, device, dl_train, dl_test, criterion, optimizer, epochs=100)
torch.save(model.state_dict(), f'models/model_basenn{time.time_ns()}.pth') # it will be saved, ...but just in case (rly doesnt make much sense)

### train phase 2 (next 300 epochs)

In [ ]:
train_loss_phase2, test_loss_phase2 = train_model(model, device, dl_train, dl_test, criterion, optimizer, epochs=300)
torch.save(model.state_dict(), f'models/model_betternn_200epochs_{time.time_ns()}.pth') # it will be saved, ...but just in case (rly doesnt make much sense)

In [ ]:
loss_y_train = np.concatenate((train_loss, train_loss_phase2))
loss_y_test = np.concatenate((test_loss, test_loss_phase2))
loss_x = np.arange(0, len(loss_y_train), 1)
# plot graph
plt.figure()
plt.plot(loss_x,loss_y_train, label='Train loss') # train loss over all epochs
plt.plot(loss_x,loss_y_test, label='Test loss') # test loss over all epochs
plt.title('Loss function')
plt.show()